# **Deep Learning Lab: MNIST with Pytorch**

__author__:Sehyun Park  
__email__:ps_hyen@snu.ac.kr

In this notebook, we construct a simple convolutional neural network with Pytorch.

In [1]:
import torch

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
device = torch.device('cuda:0') if torch.cuda.is_available() is True else None
print('Is cude avaialable (T/F): ', torch.cuda.is_available())

Is cude avaialable (T/F):  True


In [3]:
batch_size = 128
n_epoch = 3

## Make DataLoader
- The following code automatically downloads the MNIST dataset.

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.6MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.40MB/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.77MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



## Construct a convolutional neural network

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.drop1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.drop2 = nn.Dropout(0.5)   # drop rate
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.drop1(x)
        x = x.view(-1, 9216)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
model = Net().to(device) # function 'to' shift memories from CPU to GPU.
optimizer = optim.Adadelta(model.parameters())
print('model: ', model)

model:  Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (drop1): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


## Define train/test functions

In [7]:
log_interval = 100

def train(epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)

        loss.backward()
        optimizer.step(
        )
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(device):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)

        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Train the convolutional neural network!

In [8]:
for epoch in range(1, n_epoch + 1):
    train(epoch, device)
    test(device)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307725
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.198727
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.132551
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.105323
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.228460

Test set: Average loss: 0.0498, Accuracy: 9824/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.053709
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.089511
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.045720
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.054445
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.044691

Test set: Average loss: 0.0393, Accuracy: 9869/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.084908
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.073126
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.028346
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.084694
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.035482

Test set: Average loss: 0.0367, Accuracy: 9877/10000 (99%)

